In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
babs = pd.read_csv('data/201508_trip_data.csv.gz')

In [ ]:
def convert_date(date_str):
    try:
        return datetime.datetime.strptime(date_str, '%m/%d/%Y %H:%M')
    except ValueError:
        return None

In [ ]:
babs['Start Date'] = babs['Start Date'].apply(convert_date)

In [ ]:
plt.hist(babs.Duration, bins=30)
None

In [ ]:
sd, mu = babs.Duration.std(), babs.Duration.mean()

In [ ]:
babs['z'] = (babs.Duration - mu) / sd

In [ ]:
plt.hist(babs.z[babs.z < 5], bins=30)
plt.yscale('log')

In [ ]:
babs['date'] = babs['Start Date'].apply(lambda x: x.date())

In [ ]:
babs['hour'] = babs['Start Date'].apply(lambda x: x.hour)

In [ ]:
counts = babs.groupby(['date', 'hour'], as_index=False)['Trip ID'].count()

In [ ]:
X = counts.pivot_table(columns='hour', values='Trip ID', index='date').fillna(0)

In [ ]:
X.head()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
ks = range(2, 10)
scores = [-KMeans(k).fit(X).score(X) for k in ks]

In [ ]:
plt.plot(ks, scores)

In [ ]:
model = KMeans(4).fit(X)

In [ ]:
for n, center in enumerate(model.cluster_centers_):
    plt.plot(center, label="class = {}".format(n))
plt.legend()

In [ ]:
labels = pd.DataFrame({'date': X.index, 'cluster': model.labels_})

In [ ]:
from calendar import day_abbr, month_abbr

In [ ]:
labels['day'] = labels.date.apply(lambda x: day_abbr[x.weekday()])
labels['month'] = labels.date.apply(lambda x: month_abbr[x.month])

In [ ]:
weekday_counts = labels.groupby(['cluster', 'day']).cluster.count()

In [ ]:
weekday_counts

In [ ]:
weekday_counts.name = 'count'

In [ ]:
better_counts = weekday_counts.reset_index().pivot(columns='day', values='count', index='cluster').fillna(0)
better_counts

In [ ]:
better_counts.plot.bar()
plt.ylabel('count')

In [ ]:
(labels
 .groupby(['cluster', 'month'])
 .day.count().reset_index()
 .pivot(columns='month', values='day', index='cluster')
 .fillna(0)
 .plot.bar())
plt.legend(bbox_to_anchor=(1,1))
plt.ylabel('count')

In [ ]:
hours = X.columns

In [ ]:
X['center'] = [model.cluster_centers_[i] for i in model.labels_]
X['cluster'] = model.labels_

In [ ]:
from numpy.linalg import norm

In [ ]:
X['dist'] = [norm(i - j) for i, j in zip(X[hours].values, X['center'])]

In [ ]:
stats = X.groupby(model.labels_)['dist'].agg([np.mean, np.std])

In [ ]:
joined = X.join(stats, on='cluster')

In [ ]:
joined['z'] = (joined.dist - joined['mean'])/joined['std']

In [ ]:
joined.z.plot.hist()

In [ ]:
joined[joined.z > 3]

In [ ]:
for i in joined[(joined.z > 3.5) & (joined.cluster == 2)][list(hours)].iterrows():
    plt.plot(i[1], label='outlier')
plt.plot(model.cluster_centers_[2], label='center')
plt.legend()
plt.xlabel('hour')
plt.ylabel('trip count')